# Transportation problem example

|            |    Retail 1   |   Retail  2   |   Retail 3    |    Supply   |
| :-------:  | :----------:  | :----------:  | :---------:   | :---------: |
|  Supplier 1|       10      |      9        |       15      |    100      |
|  Supplier 2|       7       |      19       |       9       |    50      |
|  Supplier 3|       11      |      21       |       11      |    60      |
|  Supplier 4|       15      |      8        |       10      |    40      |
|  Demand    |      100      |     100       |       50      |            |

$$
\begin{aligned}
\min z & = 10x_{11} + 9x_{12} + 15x_{13} + \ldots + 15x_{41} + 8x_{42} + 10x_{43} \\
\text{Subjecto to:} \\
& \sum_{i=1}^{4} x_{i1} + x_{i2} + x_{i3} = \text{Supply}_i \\ 
& \sum_{j=1}^{3} x_{1j} + x_{2j} + x_{3j} + x_{4j} = \text{Demand}_j \\ 
& x_{ij} \ge 0 \\
& i = 1,2,3,4 \\
& j = 1,2,3
\end{aligned}    
$$

In [18]:
using Pkg; Pkg.activate(".")

  Activating project at `~/code/julia/notebooks`


In [19]:
using JuMP

In [20]:
using HiGHS

In [21]:
m = Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [22]:
c = [10, 7, 11, 15, 9, 19, 21, 8, 15, 9, 11, 10 ]
supply = [100, 50, 60, 40]
demand = [100, 100, 50]

3-element Vector{Int64}:
 100
 100
  50

In [23]:
sum(supply) == sum(demand)

true

In [24]:
@variable(m, x[1:4, 1:3])

4×3 Matrix{VariableRef}:
 x[1,1]  x[1,2]  x[1,3]
 x[2,1]  x[2,2]  x[2,3]
 x[3,1]  x[3,2]  x[3,3]
 x[4,1]  x[4,2]  x[4,3]

In [25]:
@objective(m, Min, sum(c .* x[:]))

10 x[1,1] + 7 x[2,1] + 11 x[3,1] + 15 x[4,1] + 9 x[1,2] + 19 x[2,2] + 21 x[3,2] + 8 x[4,2] + 15 x[1,3] + 9 x[2,3] + 11 x[3,3] + 10 x[4,3]

In [26]:
for i in 1:4
    @constraint(m, sum(x[i,:]) == supply[i])
end

In [27]:
for j in 1:3
    @constraint(m, sum(x[:,j]) == demand[j])
end

In [28]:
@constraint(m, x[:] .>= 0);

In [29]:
println(m)

Min 10 x[1,1] + 7 x[2,1] + 11 x[3,1] + 15 x[4,1] + 9 x[1,2] + 19 x[2,2] + 21 x[3,2] + 8 x[4,2] + 15 x[1,3] + 9 x[2,3] + 11 x[3,3] + 10 x[4,3]
Subject to
 x[1,1] + x[1,2] + x[1,3] = 100
 x[2,1] + x[2,2] + x[2,3] = 50
 x[3,1] + x[3,2] + x[3,3] = 60
 x[4,1] + x[4,2] + x[4,3] = 40
 x[1,1] + x[2,1] + x[3,1] + x[4,1] = 100
 x[1,2] + x[2,2] + x[3,2] + x[4,2] = 100
 x[1,3] + x[2,3] + x[3,3] + x[4,3] = 50
 x[1,1] ≥ 0
 x[2,1] ≥ 0
 x[3,1] ≥ 0
 x[4,1] ≥ 0
 x[1,2] ≥ 0
 x[2,2] ≥ 0
 x[3,2] ≥ 0
 x[4,2] ≥ 0
 x[1,3] ≥ 0
 x[2,3] ≥ 0
 x[3,3] ≥ 0
 x[4,3] ≥ 0



In [30]:
optimize!(m)

Running HiGHS 1.12.0 (git hash: 755a8e027a): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 19 rows; 12 cols; 36 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 1e+00]
  Cost    [7e+00, 2e+01]
  Bound   [0e+00, 0e+00]
  RHS     [4e+01, 1e+02]
Presolving model
7 rows, 12 cols, 24 nonzeros  0s
Dependent equations search running on 7 equations with time limit of 1000.00s
Dependent equations search removed 1 rows and 4 nonzeros in 0.00s (limit = 1000.00s)
6 rows, 12 cols, 20 nonzeros  0s
Presolve reductions: rows 6(-13); columns 12(-0); nonzeros 20(-16) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 6(400) 0s
          6     2.2700000000e+03 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 6
Objective value     :  2.2700000000e+03
P-D objective error :  0.0000000000e+00
HiGHS r

In [31]:
value.(x)

4×3 Matrix{Float64}:
 40.0  60.0  -0.0
 50.0  -0.0  -0.0
 10.0  -0.0  50.0
 -0.0  40.0  -0.0

|            |    Retail 1   |   Retail  2   |   Retail 3    |    Supply   |
| :-------:  | :----------:  | :----------:  | :---------:   | :---------: |
|  Supplier 1|       10 (40) |   9 (60)      |   15          |    100      |
|  Supplier 2|       7  (50) |   19          |       9       |    50      |
|  Supplier 3|       11 (10) |   21          |   11 (50)     |    60      |
|  Supplier 4|   15          |   8 (40)      |       10      |    40      |
|  Demand    |      100      |     100       |       50      |            |

In [32]:
totalcost = 10 * 40 + 9 * 60 + 7 * 50 + 11 * 10 + 11 * 50 + 8 * 40 

2270

In [33]:
solution_summary(m)

solution_summary(; result = 1, verbose = false)
├ solver_name          : HiGHS
├ Termination
│ ├ termination_status : OPTIMAL
│ ├ result_count       : 1
│ ├ raw_status         : kHighsModelStatusOptimal
│ └ objective_bound    : 2.27000e+03
├ Solution (result = 1)
│ ├ primal_status        : FEASIBLE_POINT
│ ├ dual_status          : FEASIBLE_POINT
│ ├ objective_value      : 2.27000e+03
│ ├ dual_objective_value : 2.27000e+03
│ └ relative_gap         : 0.00000e+00
└ Work counters
  ├ solve_time (sec)   : 3.10421e-04
  ├ simplex_iterations : 6
  ├ barrier_iterations : 0
  └ node_count         : -1

In [34]:
latex_formulation(m)

$$ \begin{aligned}
\min\quad & 10 x_{1,1} + 7 x_{2,1} + 11 x_{3,1} + 15 x_{4,1} + 9 x_{1,2} + 19 x_{2,2} + 21 x_{3,2} + 8 x_{4,2} + 15 x_{1,3} + 9 x_{2,3} + 11 x_{3,3} + 10 x_{4,3}\\
\text{Subject to} \quad & x_{1,1} + x_{1,2} + x_{1,3} = 100\\
 & x_{2,1} + x_{2,2} + x_{2,3} = 50\\
 & x_{3,1} + x_{3,2} + x_{3,3} = 60\\
 & x_{4,1} + x_{4,2} + x_{4,3} = 40\\
 & x_{1,1} + x_{2,1} + x_{3,1} + x_{4,1} = 100\\
 & x_{1,2} + x_{2,2} + x_{3,2} + x_{4,2} = 100\\
 & x_{1,3} + x_{2,3} + x_{3,3} + x_{4,3} = 50\\
 & x_{1,1} \geq 0\\
 & x_{2,1} \geq 0\\
 & x_{3,1} \geq 0\\
 & x_{4,1} \geq 0\\
 & x_{1,2} \geq 0\\
 & x_{2,2} \geq 0\\
 & x_{3,2} \geq 0\\
 & x_{4,2} \geq 0\\
 & x_{1,3} \geq 0\\
 & x_{2,3} \geq 0\\
 & x_{3,3} \geq 0\\
 & x_{4,3} \geq 0\\
\end{aligned} $$